In [1]:
from pyspark.sql import SparkSession
from delta.pip_utils import configure_spark_with_delta_pip

from nyc_taxi_eta.configs.settings import YELLOW_TAXI_DIR, BRONZE_DIR, WAREHOUSE_DIR

# Paths
SOURCE_DIR = YELLOW_TAXI_DIR

print(f"Source: {SOURCE_DIR}")
print(f"Target: {BRONZE_DIR}")

# Create Spark session with Delta Lake
spark = configure_spark_with_delta_pip(
    SparkSession.builder.appName("BronzeIngestion")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.warehouse.dir", str(WAREHOUSE_DIR))
    .config("spark.driver.memory", "80g")
    .config("spark.sql.shuffle.partitions", "12")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.parquet.compression.codec", "snappy")
).getOrCreate()

print(f"Spark version: {spark.version}")

Source: /home/administrator/Desktop/datascience/github/nyc-taxi-eta/data/landing/trips_data/yellow_taxi
Target: /home/administrator/Desktop/datascience/github/nyc-taxi-eta/data/metastore/bronze


:: loading settings :: url = jar:file:/home/administrator/Desktop/datascience/github/nyc-taxi-eta/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/administrator/.ivy2.5.2/cache
The jars for the packages stored in: /home/administrator/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-159aef77-cdf6-472e-940e-8b5929ae33a5;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 157ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|               

Spark version: 4.0.0


In [ ]:
df = spark.read.table("bronze.yellow_taxi_trips")
df.filter("year > 2019").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze`.`yellow_taxi` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01;
'UnresolvedRelation [bronze, yellow_taxi], [], false


In [5]:
# unique vendor ids to list
vendor_ids = df.select("vendor_id").distinct().rdd.flatMap(lambda x: x).collect()
print(vendor_ids)

['2', '1', '3', '5', '4', '6', '7', 'VTS', 'DDS', 'CMT']


In [7]:
spark.stop()